In [0]:
import tensorflow as tf
import numpy as np
import keras

from keras import optimizers, regularizers
from keras.datasets import cifar100
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Input
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator

# from tflearn.datasets import cifar10, cifar100

In [0]:
# Initialize parameters for the model
width = 32 # width of image in pixels
height = 32  # height of the image in pixels
n_inputs = width*height # number of pixels in one image/ number of inputs
n_classes = 100 # There are 10 possible classifications for this problem

batch_size = 128
epochs = 200
learning_rate = 0.005
reg_rate = 1.5e-4

In [0]:
class get_cifar_data(object):
    def __init__(self,n_classes):
        (xtrain,ytrain),(self.X_test,self.Y_test) = cifar100.load_data()
        self.n_classes = n_classes
        self.X_val = xtrain[:5000].astype('float32')
        self.Y_val = ytrain[:5000]
        self.X_train = xtrain[5000:].astype('float32')
        self.Y_train = ytrain[5000:]
        self.X_test = self.X_test.astype('float32')
        self.Y_test = self.Y_test
        self.Y_train = self.get_one_hot(self.Y_train,self.n_classes)
        self.Y_val = self.get_one_hot(self.Y_val,self.n_classes)
        self.Y_test = self.get_one_hot(self.Y_test,self.n_classes)

  # https://stackoverflow.com/questions/38592324/one-hot-encoding-using-numpy/38592416
    def get_one_hot(self, data,n_classes):
      targets = np.array(data).reshape(-1)
      return np.eye(n_classes)[targets]

In [0]:
class CNNModel(object):
    def __init__(self,data,batch_size,epochs, learning_rate, reg_rate):
        self.data = data
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.reg_rate = reg_rate
        self.cnn_model_fn()

In [0]:
class CNNModel(CNNModel):   
    def residual_block(self, x, depth, kernel):
        residual = x
        residual = tf.contrib.layers.batch_norm(residual, activation_fn=tf.nn.relu)
        residual = tf.contrib.layers.conv2d(residual, depth, kernel)
        residual = tf.contrib.layers.batch_norm(residual, activation_fn=tf.nn.relu)
        residual = tf.contrib.layers.conv2d(residual, x.get_shape()[3], kernel)
        residual = residual + x
        return residual
  
    def residual_network(self,
                         x,
                         num_classes=None,
                         global_pool=True,
                         output_stride=None,
                         reuse=None):
        net = x
        net = tf.contrib.layers.repeat(net, 1, tf.contrib.layers.conv2d, 32, [5, 5], stride=1)
        net = tf.contrib.layers.max_pool2d(net, [2, 2])
        net = tf.contrib.layers.repeat(net, 1, tf.contrib.layers.conv2d, 32, [5, 5], stride=1)
        net = tf.contrib.layers.max_pool2d(net, [2, 2])
        net = self.residual_block(net, 32, [3, 3])
        net = self.residual_block(net, 32, [3, 3])
        net = self.residual_block(net, 64, [3, 3])
        net = self.residual_block(net, 64, [3, 3])
        net = tf.contrib.layers.flatten(net)
        net = tf.contrib.layers.fully_connected(net, 1024)
        net = tf.contrib.layers.dropout(net, 0.5)
        net = tf.contrib.layers.fully_connected(net, num_classes, activation_fn=None, normalizer_fn=None)
        return net

In [0]:
class CNNModel(CNNModel):
    def conv_block(self,filters,dropout,input):
        x = Conv2D(filters,[3,3],strides=1,padding='same',activation='relu', kernel_regularizer=regularizers.l2(self.reg_rate))(input)
        x = BatchNormalization()(x)
        x = Conv2D(filters,[3,3],strides=1,padding='same',activation='relu', kernel_regularizer=regularizers.l2(self.reg_rate))(input)
        x = BatchNormalization()(x)
        x = MaxPooling2D(2,strides=2)(x)
        x = Dropout(dropout)(x)
        return x

    def cnn_model_fn(self):
        input = Input(shape = self.data.X_train.shape[1:])
        x = self.conv_block(32,0.25,input)        
        x = self.conv_block(64,0.35,x)
        x = self.conv_block(128,0.45,x)
        x = self.conv_block(256,0.35,x)
        x = Flatten()(x)
        x = Dense(1024,activation = 'relu')(x)
        y = Dense(self.data.n_classes,activation='softmax')(x)
        self.model = Model(inputs = input, outputs = y)

In [0]:
class CNNModel(CNNModel):    
    def LR_SCHEDULE(self, epoch):
        lr = self.learning_rate
        if epoch > 40:
            lr = self.learning_rate/20
        elif epoch > 20:
            lr = self.learning_rate/10
        return lr

    def train(self):
        self.optimizer = optimizers.Adam(self.learning_rate)
        self.model.compile(self.optimizer,'categorical_crossentropy',
                           ['accuracy', 'top_k_categorical_accuracy'])
        self.datagen = keras.preprocessing.image.ImageDataGenerator(
            horizontal_flip = True,fill_mode = 'constant',
            width_shift_range = 4, height_shift_range = 4)
        self.datagen.fit(self.data.X_train)
        self.model.fit_generator(self.datagen.flow(self.data.X_train,
            self.data.Y_train,batch_size=self.batch_size),
            steps_per_epoch=len(self.data.X_train)/self.batch_size,
            epochs=self.epochs,
            validation_data=(self.data.X_val,self.data.Y_val),
            callbacks=[LearningRateScheduler(self.LR_SCHEDULE)],verbose=2)

In [0]:
data = get_cifar_data(n_classes)
m = CNNModel(data, batch_size, epochs, learning_rate, reg_rate)
m.train()
scores = m.model.evaluate(data.X_test,data.Y_test,verbose=2)
print('\nTest Top 5 Accuracy: %.3f Test Accuracy: %.3f loss: %.3f' % (scores[2],scores[1],scores[0]))
# print('\nTest result: %.3f loss: %.3f' % (scores[1],scores[0]))

Downloading data from https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz
169009152/169001437 [==============================] - 106s 1us/step
Epoch 1/200
 - 29s - loss: 4.4296 - acc: 0.0601 - top_k_categorical_accuracy: 0.2211 - val_loss: 4.6178 - val_acc: 0.0514 - val_top_k_categorical_accuracy: 0.1914
Epoch 2/200
 - 26s - loss: 3.9007 - acc: 0.1103 - top_k_categorical_accuracy: 0.3457 - val_loss: 3.7983 - val_acc: 0.1280 - val_top_k_categorical_accuracy: 0.3922
Epoch 3/200
 - 26s - loss: 3.7359 - acc: 0.1516 - top_k_categorical_accuracy: 0.4117 - val_loss: 3.6465 - val_acc: 0.1742 - val_top_k_categorical_accuracy: 0.4466
Epoch 4/200
 - 26s - loss: 3.5747 - acc: 0.1873 - top_k_categorical_accuracy: 0.4711 - val_loss: 3.5527 - val_acc: 0.2054 - val_top_k_categorical_accuracy: 0.4878
Epoch 5/200
 - 26s - loss: 3.4791 - acc: 0.2158 - top_k_categorical_accuracy: 0.5142 - val_loss: 3.5021 - val_acc: 0.2174 - val_top_k_categorical_accuracy: 0.5120
Epoch 6/200
 - 26s - loss: 3.4140 - acc: 0.2386 - top_k_categorical_accuracy: 0.5449 - val_loss: 3.3011 - val_acc: 0.2720 - val_top_k_categorical_accuracy: 0.5826
Epoch 7/200
 - 26s - loss: 3.3545 - acc: 0.2600 - top_k_categorical_accuracy: 0.5705 - val_loss: 3.3523 - val_acc: 0.2670 - val_top_k_categorical_accuracy: 0.5724
Epoch 8/200
 - 26s - loss: 3.3217 - acc: 0.2749 - top_k_categorical_accuracy: 0.5871 - val_loss: 3.4268 - val_acc: 0.2640 - val_top_k_categorical_accuracy: 0.5666
Epoch 9/200
 - 26s - loss: 3.2865 - acc: 0.2834 - top_k_categorical_accuracy: 0.5996 - val_loss: 3.4594 - val_acc: 0.2562 - val_top_k_categorical_accuracy: 0.5676
Epoch 10/200
 - 26s - loss: 3.2557 - acc: 0.2958 - top_k_categorical_accuracy: 0.6123 - val_loss: 3.1844 - val_acc: 0.3206 - val_top_k_categorical_accuracy: 0.6238
Epoch 11/200
 - 26s - loss: 3.2198 - acc: 0.3026 - top_k_categorical_accuracy: 0.6229 - val_loss: 3.2428 - val_acc: 0.3074 - val_top_k_categorical_accuracy: 0.6268
Epoch 12/200
 - 26s - loss: 3.1891 - acc: 0.3123 - top_k_categorical_accuracy: 0.6325 - val_loss: 3.1315 - val_acc: 0.3342 - val_top_k_categorical_accuracy: 0.6500
Epoch 13/200
 - 26s - loss: 3.1881 - acc: 0.3186 - top_k_categorical_accuracy: 0.6330 - val_loss: 3.0600 - val_acc: 0.3474 - val_top_k_categorical_accuracy: 0.6648
Epoch 14/200
 - 26s - loss: 3.1598 - acc: 0.3213 - top_k_categorical_accuracy: 0.6424 - val_loss: 3.8867 - val_acc: 0.2384 - val_top_k_categorical_accuracy: 0.5634
Epoch 15/200
 - 26s - loss: 3.1488 - acc: 0.3258 - top_k_categorical_accuracy: 0.6449 - val_loss: 3.0420 - val_acc: 0.3476 - val_top_k_categorical_accuracy: 0.6778
Epoch 16/200
 - 26s - loss: 3.1243 - acc: 0.3331 - top_k_categorical_accuracy: 0.6545 - val_loss: 3.1494 - val_acc: 0.3342 - val_top_k_categorical_accuracy: 0.6538
Epoch 17/200
 - 27s - loss: 3.1021 - acc: 0.3401 - top_k_categorical_accuracy: 0.6587 - val_loss: 3.1105 - val_acc: 0.3432 - val_top_k_categorical_accuracy: 0.6670
Epoch 18/200
 - 27s - loss: 3.0878 - acc: 0.3422 - top_k_categorical_accuracy: 0.6651 - val_loss: 2.9467 - val_acc: 0.3808 - val_top_k_categorical_accuracy: 0.7000
Epoch 19/200
 - 27s - loss: 3.0758 - acc: 0.3464 - top_k_categorical_accuracy: 0.6678 - val_loss: 2.9896 - val_acc: 0.3662 - val_top_k_categorical_accuracy: 0.6840
Epoch 20/200
 - 27s - loss: 3.0718 - acc: 0.3498 - top_k_categorical_accuracy: 0.6695 - val_loss: 2.8874 - val_acc: 0.3846 - val_top_k_categorical_accuracy: 0.7046
Epoch 21/200
 - 27s - loss: 3.0564 - acc: 0.3524 - top_k_categorical_accuracy: 0.6746 - val_loss: 2.8980 - val_acc: 0.3922 - val_top_k_categorical_accuracy: 0.7042
Epoch 22/200
 - 27s - loss: 2.7993 - acc: 0.4032 - top_k_categorical_accuracy: 0.7228 - val_loss: 2.6955 - val_acc: 0.4320 - val_top_k_categorical_accuracy: 0.7372
Epoch 23/200
 - 27s - loss: 2.6727 - acc: 0.4198 - top_k_categorical_accuracy: 0.7375 - val_loss: 2.6968 - val_acc: 0.4166 - val_top_k_categorical_accuracy: 0.7338
Epoch 24/200
 - 27s - loss: 2.6025 - acc: 0.4274 - top_k_categorical_accuracy: 0.7428 - val_loss: 2.5644 - val_acc: 0.4398 - val_top_k_categorical_accuracy: 0.7494
Epoch 25/200
 - 26s - loss: 2.5482 - acc: 0.4331 - top_k_categorical_accuracy: 0.7481 - val_loss: 2.5574 - val_acc: 0.4376 - val_top_k_categorical_accuracy: 0.7496
Epoch 26/200
 - 27s - loss: 2.4959 - acc: 0.4390 - top_k_categorical_accuracy: 0.7558 - val_loss: 2.6106 - val_acc: 0.4184 - val_top_k_categorical_accuracy: 0.7282
Epoch 27/200
 - 27s - loss: 2.4481 - acc: 0.4450 - top_k_categorical_accuracy: 0.7596 - val_loss: 2.4669 - val_acc: 0.4420 - val_top_k_categorical_accuracy: 0.7552
Epoch 28/200
 - 27s - loss: 2.4145 - acc: 0.4439 - top_k_categorical_accuracy: 0.7627 - val_loss: 2.4614 - val_acc: 0.4408 - val_top_k_categorical_accuracy: 0.7534
Epoch 29/200
 - 27s - loss: 2.3789 - acc: 0.4491 - top_k_categorical_accuracy: 0.7662 - val_loss: 2.3896 - val_acc: 0.4512 - val_top_k_categorical_accuracy: 0.7666
Epoch 30/200
 - 27s - loss: 2.3400 - acc: 0.4566 - top_k_categorical_accuracy: 0.7704 - val_loss: 2.3419 - val_acc: 0.4576 - val_top_k_categorical_accuracy: 0.7666
Epoch 31/200
 - 26s - loss: 2.3139 - acc: 0.4600 - top_k_categorical_accuracy: 0.7721 - val_loss: 2.3525 - val_acc: 0.4544 - val_top_k_categorical_accuracy: 0.7620
Epoch 32/200
 - 27s - loss: 2.2977 - acc: 0.4599 - top_k_categorical_accuracy: 0.7741 - val_loss: 2.4800 - val_acc: 0.4286 - val_top_k_categorical_accuracy: 0.7408
Epoch 33/200
 - 27s - loss: 2.2635 - acc: 0.4638 - top_k_categorical_accuracy: 0.7758 - val_loss: 2.4126 - val_acc: 0.4348 - val_top_k_categorical_accuracy: 0.7536
Epoch 34/200
 - 27s - loss: 2.2460 - acc: 0.4668 - top_k_categorical_accuracy: 0.7786 - val_loss: 2.3586 - val_acc: 0.4430 - val_top_k_categorical_accuracy: 0.7586
Epoch 35/200
 - 27s - loss: 2.2214 - acc: 0.4674 - top_k_categorical_accuracy: 0.7813 - val_loss: 2.3955 - val_acc: 0.4400 - val_top_k_categorical_accuracy: 0.7512
Epoch 36/200
 - 27s - loss: 2.1979 - acc: 0.4717 - top_k_categorical_accuracy: 0.7852 - val_loss: 2.2409 - val_acc: 0.4688 - val_top_k_categorical_accuracy: 0.7816
Epoch 37/200
 - 27s - loss: 2.1886 - acc: 0.4751 - top_k_categorical_accuracy: 0.7848 - val_loss: 2.2510 - val_acc: 0.4626 - val_top_k_categorical_accuracy: 0.7768
Epoch 38/200
 - 27s - loss: 2.1655 - acc: 0.4778 - top_k_categorical_accuracy: 0.7864 - val_loss: 2.2271 - val_acc: 0.4736 - val_top_k_categorical_accuracy: 0.7802
Epoch 39/200
 - 27s - loss: 2.1520 - acc: 0.4797 - top_k_categorical_accuracy: 0.7890 - val_loss: 2.2039 - val_acc: 0.4744 - val_top_k_categorical_accuracy: 0.7812
Epoch 40/200
 - 27s - loss: 2.1364 - acc: 0.4834 - top_k_categorical_accuracy: 0.7917 - val_loss: 2.2376 - val_acc: 0.4662 - val_top_k_categorical_accuracy: 0.7764
Epoch 41/200
 - 27s - loss: 2.1141 - acc: 0.4856 - top_k_categorical_accuracy: 0.7943 - val_loss: 2.3854 - val_acc: 0.4394 - val_top_k_categorical_accuracy: 0.7538
Epoch 42/200
 - 27s - loss: 2.0765 - acc: 0.4936 - top_k_categorical_accuracy: 0.8009 - val_loss: 2.1913 - val_acc: 0.4814 - val_top_k_categorical_accuracy: 0.7818
Epoch 43/200
 - 27s - loss: 2.0569 - acc: 0.4964 - top_k_categorical_accuracy: 0.8038 - val_loss: 2.1925 - val_acc: 0.4804 - val_top_k_categorical_accuracy: 0.7826
Epoch 44/200
 - 27s - loss: 2.0520 - acc: 0.4965 - top_k_categorical_accuracy: 0.8025 - val_loss: 2.1876 - val_acc: 0.4768 - val_top_k_categorical_accuracy: 0.7850
Epoch 45/200
 - 27s - loss: 2.0399 - acc: 0.4973 - top_k_categorical_accuracy: 0.8068 - val_loss: 2.2194 - val_acc: 0.4756 - val_top_k_categorical_accuracy: 0.7812
Epoch 46/200
 - 27s - loss: 2.0271 - acc: 0.5010 - top_k_categorical_accuracy: 0.8060 - val_loss: 2.2180 - val_acc: 0.4740 - val_top_k_categorical_accuracy: 0.7756
Epoch 47/200
 - 26s - loss: 2.0141 - acc: 0.5047 - top_k_categorical_accuracy: 0.8086 - val_loss: 2.2393 - val_acc: 0.4710 - val_top_k_categorical_accuracy: 0.7770
Epoch 48/200
 - 27s - loss: 2.0072 - acc: 0.5032 - top_k_categorical_accuracy: 0.8093 - val_loss: 2.1267 - val_acc: 0.4918 - val_top_k_categorical_accuracy: 0.7926
Epoch 49/200
 - 27s - loss: 1.9931 - acc: 0.5046 - top_k_categorical_accuracy: 0.8118 - val_loss: 2.1279 - val_acc: 0.4946 - val_top_k_categorical_accuracy: 0.7908
Epoch 50/200
 - 27s - loss: 1.9835 - acc: 0.5086 - top_k_categorical_accuracy: 0.8137 - val_loss: 2.1772 - val_acc: 0.4818 - val_top_k_categorical_accuracy: 0.7832
Epoch 51/200
 - 27s - loss: 1.9786 - acc: 0.5094 - top_k_categorical_accuracy: 0.8118 - val_loss: 2.1677 - val_acc: 0.4842 - val_top_k_categorical_accuracy: 0.7850
Epoch 52/200
 - 27s - loss: 1.9574 - acc: 0.5139 - top_k_categorical_accuracy: 0.8162 - val_loss: 2.1133 - val_acc: 0.4964 - val_top_k_categorical_accuracy: 0.7904
Epoch 53/200
 - 27s - loss: 1.9600 - acc: 0.5140 - top_k_categorical_accuracy: 0.8166 - val_loss: 2.1196 - val_acc: 0.4946 - val_top_k_categorical_accuracy: 0.7910
Epoch 54/200
 - 27s - loss: 1.9404 - acc: 0.5178 - top_k_categorical_accuracy: 0.8199 - val_loss: 2.1755 - val_acc: 0.4842 - val_top_k_categorical_accuracy: 0.7820
Epoch 55/200
 - 27s - loss: 1.9415 - acc: 0.5180 - top_k_categorical_accuracy: 0.8165 - val_loss: 2.1706 - val_acc: 0.4834 - val_top_k_categorical_accuracy: 0.7820
Epoch 56/200
 - 27s - loss: 1.9184 - acc: 0.5224 - top_k_categorical_accuracy: 0.8218 - val_loss: 2.2048 - val_acc: 0.4782 - val_top_k_categorical_accuracy: 0.7780
Epoch 57/200
 - 27s - loss: 1.9262 - acc: 0.5180 - top_k_categorical_accuracy: 0.8197 - val_loss: 2.1511 - val_acc: 0.4862 - val_top_k_categorical_accuracy: 0.7872
Epoch 58/200
 - 27s - loss: 1.9076 - acc: 0.5214 - top_k_categorical_accuracy: 0.8225 - val_loss: 2.1508 - val_acc: 0.4870 - val_top_k_categorical_accuracy: 0.7854
Epoch 59/200
 - 27s - loss: 1.8907 - acc: 0.5254 - top_k_categorical_accuracy: 0.8274 - val_loss: 2.0745 - val_acc: 0.4962 - val_top_k_categorical_accuracy: 0.7994
Epoch 60/200
 - 27s - loss: 1.8920 - acc: 0.5255 - top_k_categorical_accuracy: 0.8260 - val_loss: 2.0925 - val_acc: 0.4978 - val_top_k_categorical_accuracy: 0.7942
Epoch 61/200
 - 27s - loss: 1.8855 - acc: 0.5228 - top_k_categorical_accuracy: 0.8269 - val_loss: 2.1504 - val_acc: 0.4788 - val_top_k_categorical_accuracy: 0.7870
Epoch 62/200
 - 27s - loss: 1.8734 - acc: 0.5272 - top_k_categorical_accuracy: 0.8281 - val_loss: 2.1784 - val_acc: 0.4746 - val_top_k_categorical_accuracy: 0.7800
Epoch 63/200
 - 27s - loss: 1.8651 - acc: 0.5299 - top_k_categorical_accuracy: 0.8297 - val_loss: 2.1146 - val_acc: 0.4918 - val_top_k_categorical_accuracy: 0.7914
Epoch 64/200
 - 27s - loss: 1.8552 - acc: 0.5338 - top_k_categorical_accuracy: 0.8287 - val_loss: 2.1948 - val_acc: 0.4708 - val_top_k_categorical_accuracy: 0.7772
Epoch 65/200
 - 27s - loss: 1.8537 - acc: 0.5318 - top_k_categorical_accuracy: 0.8299 - val_loss: 2.1267 - val_acc: 0.4848 - val_top_k_categorical_accuracy: 0.7864
Epoch 66/200
 - 26s - loss: 1.8410 - acc: 0.5358 - top_k_categorical_accuracy: 0.8328 - val_loss: 2.0807 - val_acc: 0.4982 - val_top_k_categorical_accuracy: 0.7980
Epoch 67/200
 - 27s - loss: 1.8388 - acc: 0.5346 - top_k_categorical_accuracy: 0.8307 - val_loss: 2.2050 - val_acc: 0.4728 - val_top_k_categorical_accuracy: 0.7736
Epoch 68/200
 - 27s - loss: 1.8271 - acc: 0.5364 - top_k_categorical_accuracy: 0.8368 - val_loss: 2.1015 - val_acc: 0.4980 - val_top_k_categorical_accuracy: 0.7956
Epoch 69/200
 - 27s - loss: 1.8131 - acc: 0.5401 - top_k_categorical_accuracy: 0.8355 - val_loss: 2.1512 - val_acc: 0.4870 - val_top_k_categorical_accuracy: 0.7812
Epoch 70/200
 - 27s - loss: 1.8199 - acc: 0.5365 - top_k_categorical_accuracy: 0.8348 - val_loss: 2.0844 - val_acc: 0.5014 - val_top_k_categorical_accuracy: 0.7954
Epoch 71/200
 - 27s - loss: 1.8071 - acc: 0.5390 - top_k_categorical_accuracy: 0.8370 - val_loss: 2.0415 - val_acc: 0.5112 - val_top_k_categorical_accuracy: 0.7996
Epoch 72/200
 - 27s - loss: 1.7951 - acc: 0.5402 - top_k_categorical_accuracy: 0.8412 - val_loss: 2.1678 - val_acc: 0.4784 - val_top_k_categorical_accuracy: 0.7844
Epoch 73/200
 - 27s - loss: 1.7955 - acc: 0.5416 - top_k_categorical_accuracy: 0.8396 - val_loss: 2.0708 - val_acc: 0.5034 - val_top_k_categorical_accuracy: 0.7966
Epoch 74/200
 - 27s - loss: 1.7805 - acc: 0.5468 - top_k_categorical_accuracy: 0.8396 - val_loss: 2.1149 - val_acc: 0.4934 - val_top_k_categorical_accuracy: 0.7924
Epoch 75/200
 - 27s - loss: 1.7809 - acc: 0.5465 - top_k_categorical_accuracy: 0.8412 - val_loss: 2.0673 - val_acc: 0.5044 - val_top_k_categorical_accuracy: 0.7976
Epoch 76/200
 - 27s - loss: 1.7727 - acc: 0.5454 - top_k_categorical_accuracy: 0.8433 - val_loss: 2.1144 - val_acc: 0.4938 - val_top_k_categorical_accuracy: 0.7906
Epoch 77/200
 - 27s - loss: 1.7609 - acc: 0.5506 - top_k_categorical_accuracy: 0.8441 - val_loss: 2.0648 - val_acc: 0.5084 - val_top_k_categorical_accuracy: 0.7972
Epoch 78/200
 - 27s - loss: 1.7597 - acc: 0.5544 - top_k_categorical_accuracy: 0.8444 - val_loss: 2.0818 - val_acc: 0.5006 - val_top_k_categorical_accuracy: 0.7930
Epoch 79/200
 - 27s - loss: 1.7450 - acc: 0.5534 - top_k_categorical_accuracy: 0.8485 - val_loss: 2.1121 - val_acc: 0.4910 - val_top_k_categorical_accuracy: 0.7912
Epoch 80/200
 - 27s - loss: 1.7448 - acc: 0.5551 - top_k_categorical_accuracy: 0.8447 - val_loss: 2.0530 - val_acc: 0.5046 - val_top_k_categorical_accuracy: 0.7988
Epoch 81/200
 - 27s - loss: 1.7311 - acc: 0.5552 - top_k_categorical_accuracy: 0.8487 - val_loss: 2.2109 - val_acc: 0.4728 - val_top_k_categorical_accuracy: 0.7774
Epoch 82/200
 - 27s - loss: 1.7288 - acc: 0.5550 - top_k_categorical_accuracy: 0.8494 - val_loss: 2.0768 - val_acc: 0.5030 - val_top_k_categorical_accuracy: 0.7956
Epoch 83/200
 - 26s - loss: 1.7216 - acc: 0.5539 - top_k_categorical_accuracy: 0.8504 - val_loss: 2.0516 - val_acc: 0.5066 - val_top_k_categorical_accuracy: 0.8016
Epoch 84/200
 - 27s - loss: 1.7217 - acc: 0.5571 - top_k_categorical_accuracy: 0.8500 - val_loss: 2.2194 - val_acc: 0.4714 - val_top_k_categorical_accuracy: 0.7782
Epoch 85/200
 - 27s - loss: 1.7128 - acc: 0.5593 - top_k_categorical_accuracy: 0.8510 - val_loss: 2.1057 - val_acc: 0.4950 - val_top_k_categorical_accuracy: 0.7928
Epoch 86/200
 - 27s - loss: 1.7117 - acc: 0.5585 - top_k_categorical_accuracy: 0.8516 - val_loss: 2.1484 - val_acc: 0.4830 - val_top_k_categorical_accuracy: 0.7896
Epoch 87/200
 - 27s - loss: 1.7013 - acc: 0.5601 - top_k_categorical_accuracy: 0.8534 - val_loss: 2.0344 - val_acc: 0.5126 - val_top_k_categorical_accuracy: 0.8040
Epoch 88/200
 - 27s - loss: 1.6868 - acc: 0.5669 - top_k_categorical_accuracy: 0.8557 - val_loss: 2.0516 - val_acc: 0.5052 - val_top_k_categorical_accuracy: 0.7982
Epoch 89/200
 - 27s - loss: 1.6917 - acc: 0.5636 - top_k_categorical_accuracy: 0.8550 - val_loss: 2.0430 - val_acc: 0.5098 - val_top_k_categorical_accuracy: 0.8028
Epoch 90/200
 - 27s - loss: 1.6817 - acc: 0.5635 - top_k_categorical_accuracy: 0.8559 - val_loss: 2.0955 - val_acc: 0.4992 - val_top_k_categorical_accuracy: 0.7920
Epoch 91/200
 - 27s - loss: 1.6753 - acc: 0.5648 - top_k_categorical_accuracy: 0.8571 - val_loss: 2.0395 - val_acc: 0.5108 - val_top_k_categorical_accuracy: 0.8018
Epoch 92/200
 - 27s - loss: 1.6651 - acc: 0.5702 - top_k_categorical_accuracy: 0.8594 - val_loss: 2.0057 - val_acc: 0.5148 - val_top_k_categorical_accuracy: 0.8110
Epoch 93/200
 - 27s - loss: 1.6651 - acc: 0.5677 - top_k_categorical_accuracy: 0.8579 - val_loss: 2.0444 - val_acc: 0.5124 - val_top_k_categorical_accuracy: 0.7992
Epoch 94/200
 - 27s - loss: 1.6574 - acc: 0.5710 - top_k_categorical_accuracy: 0.8601 - val_loss: 2.0231 - val_acc: 0.5152 - val_top_k_categorical_accuracy: 0.8018
Epoch 95/200
 - 27s - loss: 1.6542 - acc: 0.5714 - top_k_categorical_accuracy: 0.8609 - val_loss: 2.0476 - val_acc: 0.5116 - val_top_k_categorical_accuracy: 0.7990
Epoch 96/200
 - 27s - loss: 1.6480 - acc: 0.5735 - top_k_categorical_accuracy: 0.8600 - val_loss: 2.0381 - val_acc: 0.5130 - val_top_k_categorical_accuracy: 0.8054
Epoch 97/200
 - 27s - loss: 1.6389 - acc: 0.5739 - top_k_categorical_accuracy: 0.8624 - val_loss: 2.1662 - val_acc: 0.4894 - val_top_k_categorical_accuracy: 0.7852
Epoch 98/200
 - 27s - loss: 1.6302 - acc: 0.5765 - top_k_categorical_accuracy: 0.8656 - val_loss: 2.1048 - val_acc: 0.5018 - val_top_k_categorical_accuracy: 0.7942
Epoch 99/200
 - 27s - loss: 1.6229 - acc: 0.5779 - top_k_categorical_accuracy: 0.8642 - val_loss: 2.0980 - val_acc: 0.5046 - val_top_k_categorical_accuracy: 0.7920
Epoch 100/200
 - 27s - loss: 1.6208 - acc: 0.5784 - top_k_categorical_accuracy: 0.8646 - val_loss: 2.0374 - val_acc: 0.5172 - val_top_k_categorical_accuracy: 0.8018
Epoch 101/200
 - 27s - loss: 1.6302 - acc: 0.5767 - top_k_categorical_accuracy: 0.8655 - val_loss: 2.1035 - val_acc: 0.5004 - val_top_k_categorical_accuracy: 0.7932
Epoch 102/200
 - 27s - loss: 1.6092 - acc: 0.5818 - top_k_categorical_accuracy: 0.8675 - val_loss: 1.9833 - val_acc: 0.5232 - val_top_k_categorical_accuracy: 0.8098
Epoch 103/200
 - 27s - loss: 1.6125 - acc: 0.5787 - top_k_categorical_accuracy: 0.8685 - val_loss: 2.0373 - val_acc: 0.5084 - val_top_k_categorical_accuracy: 0.8032
Epoch 104/200
 - 27s - loss: 1.6019 - acc: 0.5842 - top_k_categorical_accuracy: 0.8663 - val_loss: 2.0678 - val_acc: 0.5026 - val_top_k_categorical_accuracy: 0.8000
Epoch 105/200
 - 27s - loss: 1.5996 - acc: 0.5840 - top_k_categorical_accuracy: 0.8702 - val_loss: 2.0688 - val_acc: 0.5114 - val_top_k_categorical_accuracy: 0.7984
Epoch 106/200
 - 26s - loss: 1.5812 - acc: 0.5854 - top_k_categorical_accuracy: 0.8726 - val_loss: 2.0403 - val_acc: 0.5218 - val_top_k_categorical_accuracy: 0.8058
Epoch 107/200
 - 27s - loss: 1.5779 - acc: 0.5867 - top_k_categorical_accuracy: 0.8731 - val_loss: 2.0297 - val_acc: 0.5186 - val_top_k_categorical_accuracy: 0.8060
Epoch 108/200
 - 26s - loss: 1.5883 - acc: 0.5845 - top_k_categorical_accuracy: 0.8690 - val_loss: 2.0082 - val_acc: 0.5214 - val_top_k_categorical_accuracy: 0.8062
Epoch 109/200
 - 27s - loss: 1.5818 - acc: 0.5854 - top_k_categorical_accuracy: 0.8721 - val_loss: 2.0327 - val_acc: 0.5242 - val_top_k_categorical_accuracy: 0.8072
Epoch 110/200
 - 27s - loss: 1.5709 - acc: 0.5900 - top_k_categorical_accuracy: 0.8739 - val_loss: 2.0086 - val_acc: 0.5238 - val_top_k_categorical_accuracy: 0.8116
Epoch 111/200
 - 27s - loss: 1.5724 - acc: 0.5884 - top_k_categorical_accuracy: 0.8737 - val_loss: 2.0523 - val_acc: 0.5140 - val_top_k_categorical_accuracy: 0.8032
Epoch 112/200
 - 27s - loss: 1.5552 - acc: 0.5924 - top_k_categorical_accuracy: 0.8739 - val_loss: 2.0258 - val_acc: 0.5214 - val_top_k_categorical_accuracy: 0.8050
Epoch 113/200
 - 27s - loss: 1.5444 - acc: 0.5946 - top_k_categorical_accuracy: 0.8762 - val_loss: 2.0120 - val_acc: 0.5242 - val_top_k_categorical_accuracy: 0.8110
Epoch 114/200
 - 27s - loss: 1.5454 - acc: 0.5949 - top_k_categorical_accuracy: 0.8761 - val_loss: 2.0499 - val_acc: 0.5154 - val_top_k_categorical_accuracy: 0.8050
Epoch 115/200
 - 26s - loss: 1.5434 - acc: 0.5964 - top_k_categorical_accuracy: 0.8773 - val_loss: 2.0240 - val_acc: 0.5296 - val_top_k_categorical_accuracy: 0.8040
Epoch 116/200
 - 27s - loss: 1.5364 - acc: 0.5950 - top_k_categorical_accuracy: 0.8783 - val_loss: 2.0593 - val_acc: 0.5166 - val_top_k_categorical_accuracy: 0.8028
Epoch 117/200
 - 27s - loss: 1.5239 - acc: 0.6001 - top_k_categorical_accuracy: 0.8801 - val_loss: 2.0589 - val_acc: 0.5146 - val_top_k_categorical_accuracy: 0.7980
Epoch 118/200
 - 27s - loss: 1.5327 - acc: 0.5997 - top_k_categorical_accuracy: 0.8798 - val_loss: 2.0352 - val_acc: 0.5212 - val_top_k_categorical_accuracy: 0.8028
Epoch 119/200
 - 26s - loss: 1.5122 - acc: 0.6010 - top_k_categorical_accuracy: 0.8814 - val_loss: 2.0402 - val_acc: 0.5228 - val_top_k_categorical_accuracy: 0.8040
Epoch 120/200
 - 27s - loss: 1.5143 - acc: 0.6037 - top_k_categorical_accuracy: 0.8813 - val_loss: 2.0717 - val_acc: 0.5214 - val_top_k_categorical_accuracy: 0.8016
Epoch 121/200
 - 27s - loss: 1.4991 - acc: 0.6052 - top_k_categorical_accuracy: 0.8835 - val_loss: 2.0220 - val_acc: 0.5220 - val_top_k_categorical_accuracy: 0.8088
Epoch 122/200
 - 27s - loss: 1.5040 - acc: 0.6034 - top_k_categorical_accuracy: 0.8837 - val_loss: 2.1340 - val_acc: 0.5042 - val_top_k_categorical_accuracy: 0.7928
Epoch 123/200
 - 27s - loss: 1.4955 - acc: 0.6049 - top_k_categorical_accuracy: 0.8861 - val_loss: 2.1099 - val_acc: 0.5110 - val_top_k_categorical_accuracy: 0.7996
Epoch 124/200
 - 27s - loss: 1.4949 - acc: 0.6054 - top_k_categorical_accuracy: 0.8824 - val_loss: 2.0681 - val_acc: 0.5172 - val_top_k_categorical_accuracy: 0.8046
Epoch 125/200
 - 27s - loss: 1.4899 - acc: 0.6061 - top_k_categorical_accuracy: 0.8850 - val_loss: 2.0907 - val_acc: 0.5132 - val_top_k_categorical_accuracy: 0.7974
Epoch 126/200
 - 27s - loss: 1.4850 - acc: 0.6098 - top_k_categorical_accuracy: 0.8846 - val_loss: 2.0258 - val_acc: 0.5236 - val_top_k_categorical_accuracy: 0.8064
Epoch 127/200
 - 27s - loss: 1.4814 - acc: 0.6105 - top_k_categorical_accuracy: 0.8864 - val_loss: 2.0195 - val_acc: 0.5246 - val_top_k_categorical_accuracy: 0.8062
Epoch 128/200
 - 27s - loss: 1.4768 - acc: 0.6095 - top_k_categorical_accuracy: 0.8881 - val_loss: 2.0304 - val_acc: 0.5270 - val_top_k_categorical_accuracy: 0.8094
Epoch 129/200
 - 27s - loss: 1.4663 - acc: 0.6101 - top_k_categorical_accuracy: 0.8882 - val_loss: 2.1323 - val_acc: 0.5096 - val_top_k_categorical_accuracy: 0.7960
Epoch 130/200
 - 27s - loss: 1.4683 - acc: 0.6130 - top_k_categorical_accuracy: 0.8877 - val_loss: 2.0490 - val_acc: 0.5236 - val_top_k_categorical_accuracy: 0.8082
Epoch 131/200
 - 27s - loss: 1.4524 - acc: 0.6167 - top_k_categorical_accuracy: 0.8915 - val_loss: 2.0299 - val_acc: 0.5266 - val_top_k_categorical_accuracy: 0.8072
Epoch 132/200
 - 26s - loss: 1.4592 - acc: 0.6145 - top_k_categorical_accuracy: 0.8881 - val_loss: 2.0901 - val_acc: 0.5172 - val_top_k_categorical_accuracy: 0.8016
Epoch 133/200
 - 27s - loss: 1.4541 - acc: 0.6125 - top_k_categorical_accuracy: 0.8890 - val_loss: 2.0244 - val_acc: 0.5246 - val_top_k_categorical_accuracy: 0.8086
Epoch 134/200
 - 27s - loss: 1.4550 - acc: 0.6138 - top_k_categorical_accuracy: 0.8910 - val_loss: 2.0167 - val_acc: 0.5234 - val_top_k_categorical_accuracy: 0.8104
Epoch 135/200
 - 27s - loss: 1.4495 - acc: 0.6175 - top_k_categorical_accuracy: 0.8913 - val_loss: 2.1129 - val_acc: 0.5108 - val_top_k_categorical_accuracy: 0.7988
Epoch 136/200
 - 27s - loss: 1.4447 - acc: 0.6175 - top_k_categorical_accuracy: 0.8912 - val_loss: 2.0367 - val_acc: 0.5200 - val_top_k_categorical_accuracy: 0.8098
Epoch 137/200
 - 27s - loss: 1.4348 - acc: 0.6195 - top_k_categorical_accuracy: 0.8925 - val_loss: 2.1393 - val_acc: 0.5050 - val_top_k_categorical_accuracy: 0.7946
Epoch 138/200
 - 27s - loss: 1.4357 - acc: 0.6194 - top_k_categorical_accuracy: 0.8939 - val_loss: 2.0965 - val_acc: 0.5140 - val_top_k_categorical_accuracy: 0.8040
Epoch 139/200
 - 27s - loss: 1.4242 - acc: 0.6225 - top_k_categorical_accuracy: 0.8945 - val_loss: 2.0480 - val_acc: 0.5208 - val_top_k_categorical_accuracy: 0.8022
Epoch 140/200
 - 27s - loss: 1.4295 - acc: 0.6212 - top_k_categorical_accuracy: 0.8937 - val_loss: 2.1187 - val_acc: 0.5110 - val_top_k_categorical_accuracy: 0.7956
Epoch 141/200
 - 27s - loss: 1.4182 - acc: 0.6238 - top_k_categorical_accuracy: 0.8951 - val_loss: 2.0320 - val_acc: 0.5290 - val_top_k_categorical_accuracy: 0.8100
Epoch 142/200
 - 27s - loss: 1.4149 - acc: 0.6259 - top_k_categorical_accuracy: 0.8967 - val_loss: 2.1050 - val_acc: 0.5140 - val_top_k_categorical_accuracy: 0.8054
Epoch 143/200
 - 27s - loss: 1.4082 - acc: 0.6249 - top_k_categorical_accuracy: 0.8976 - val_loss: 2.1423 - val_acc: 0.5088 - val_top_k_categorical_accuracy: 0.7954
Epoch 144/200
 - 27s - loss: 1.4064 - acc: 0.6250 - top_k_categorical_accuracy: 0.8981 - val_loss: 2.0674 - val_acc: 0.5184 - val_top_k_categorical_accuracy: 0.8098
Epoch 145/200
 - 27s - loss: 1.4063 - acc: 0.6254 - top_k_categorical_accuracy: 0.8974 - val_loss: 2.0467 - val_acc: 0.5270 - val_top_k_categorical_accuracy: 0.8084
Epoch 146/200
 - 27s - loss: 1.4073 - acc: 0.6268 - top_k_categorical_accuracy: 0.8973 - val_loss: 2.1108 - val_acc: 0.5118 - val_top_k_categorical_accuracy: 0.7984
Epoch 147/200
 - 27s - loss: 1.3976 - acc: 0.6252 - top_k_categorical_accuracy: 0.8994 - val_loss: 2.1209 - val_acc: 0.5164 - val_top_k_categorical_accuracy: 0.7990
Epoch 148/200
 - 27s - loss: 1.3988 - acc: 0.6283 - top_k_categorical_accuracy: 0.8983 - val_loss: 2.0491 - val_acc: 0.5212 - val_top_k_categorical_accuracy: 0.8054
Epoch 149/200
 - 27s - loss: 1.3851 - acc: 0.6299 - top_k_categorical_accuracy: 0.9004 - val_loss: 2.0319 - val_acc: 0.5298 - val_top_k_categorical_accuracy: 0.8110
Epoch 150/200
 - 27s - loss: 1.3854 - acc: 0.6298 - top_k_categorical_accuracy: 0.9009 - val_loss: 2.1252 - val_acc: 0.5124 - val_top_k_categorical_accuracy: 0.8020
Epoch 151/200
 - 26s - loss: 1.3819 - acc: 0.6302 - top_k_categorical_accuracy: 0.9016 - val_loss: 2.1140 - val_acc: 0.5180 - val_top_k_categorical_accuracy: 0.8016
Epoch 152/200
 - 27s - loss: 1.3757 - acc: 0.6334 - top_k_categorical_accuracy: 0.9018 - val_loss: 2.0648 - val_acc: 0.5230 - val_top_k_categorical_accuracy: 0.8070
Epoch 153/200
 - 27s - loss: 1.3692 - acc: 0.6339 - top_k_categorical_accuracy: 0.9034 - val_loss: 2.1183 - val_acc: 0.5122 - val_top_k_categorical_accuracy: 0.7992
Epoch 154/200
 - 27s - loss: 1.3765 - acc: 0.6330 - top_k_categorical_accuracy: 0.9025 - val_loss: 2.0787 - val_acc: 0.5234 - val_top_k_categorical_accuracy: 0.8096
Epoch 155/200
 - 27s - loss: 1.3713 - acc: 0.6329 - top_k_categorical_accuracy: 0.9038 - val_loss: 2.0736 - val_acc: 0.5178 - val_top_k_categorical_accuracy: 0.8072
Epoch 156/200
 - 27s - loss: 1.3606 - acc: 0.6372 - top_k_categorical_accuracy: 0.9030 - val_loss: 2.1156 - val_acc: 0.5170 - val_top_k_categorical_accuracy: 0.8004
Epoch 157/200
 - 27s - loss: 1.3561 - acc: 0.6368 - top_k_categorical_accuracy: 0.9055 - val_loss: 2.0630 - val_acc: 0.5266 - val_top_k_categorical_accuracy: 0.8084
Epoch 158/200
 - 26s - loss: 1.3492 - acc: 0.6403 - top_k_categorical_accuracy: 0.9050 - val_loss: 2.1566 - val_acc: 0.5158 - val_top_k_categorical_accuracy: 0.7960
Epoch 159/200
 - 27s - loss: 1.3538 - acc: 0.6375 - top_k_categorical_accuracy: 0.9053 - val_loss: 2.0901 - val_acc: 0.5188 - val_top_k_categorical_accuracy: 0.8052
Epoch 160/200
 - 27s - loss: 1.3355 - acc: 0.6442 - top_k_categorical_accuracy: 0.9085 - val_loss: 2.0679 - val_acc: 0.5240 - val_top_k_categorical_accuracy: 0.8092
Epoch 161/200
 - 27s - loss: 1.3401 - acc: 0.6420 - top_k_categorical_accuracy: 0.9069 - val_loss: 2.0686 - val_acc: 0.5180 - val_top_k_categorical_accuracy: 0.8074
Epoch 162/200
 - 27s - loss: 1.3483 - acc: 0.6394 - top_k_categorical_accuracy: 0.9069 - val_loss: 2.0969 - val_acc: 0.5150 - val_top_k_categorical_accuracy: 0.8044
Epoch 163/200
 - 27s - loss: 1.3340 - acc: 0.6418 - top_k_categorical_accuracy: 0.9068 - val_loss: 2.1086 - val_acc: 0.5138 - val_top_k_categorical_accuracy: 0.8084
Epoch 164/200
 - 27s - loss: 1.3359 - acc: 0.6399 - top_k_categorical_accuracy: 0.9080 - val_loss: 2.1182 - val_acc: 0.5176 - val_top_k_categorical_accuracy: 0.8086
Epoch 165/200
 - 27s - loss: 1.3249 - acc: 0.6442 - top_k_categorical_accuracy: 0.9103 - val_loss: 2.1104 - val_acc: 0.5232 - val_top_k_categorical_accuracy: 0.8066
Epoch 166/200
 - 27s - loss: 1.3200 - acc: 0.6469 - top_k_categorical_accuracy: 0.9095 - val_loss: 2.1183 - val_acc: 0.5140 - val_top_k_categorical_accuracy: 0.8028
Epoch 167/200
 - 27s - loss: 1.3238 - acc: 0.6481 - top_k_categorical_accuracy: 0.9085 - val_loss: 2.0844 - val_acc: 0.5224 - val_top_k_categorical_accuracy: 0.8064
Epoch 168/200
 - 28s - loss: 1.3095 - acc: 0.6488 - top_k_categorical_accuracy: 0.9119 - val_loss: 2.0795 - val_acc: 0.5220 - val_top_k_categorical_accuracy: 0.8084
Epoch 169/200
 - 27s - loss: 1.3154 - acc: 0.6461 - top_k_categorical_accuracy: 0.9125 - val_loss: 2.1704 - val_acc: 0.5142 - val_top_k_categorical_accuracy: 0.7992
Epoch 170/200
 - 27s - loss: 1.3116 - acc: 0.6492 - top_k_categorical_accuracy: 0.9122 - val_loss: 2.1163 - val_acc: 0.5160 - val_top_k_categorical_accuracy: 0.8050
Epoch 171/200
 - 27s - loss: 1.3046 - acc: 0.6507 - top_k_categorical_accuracy: 0.9130 - val_loss: 2.1075 - val_acc: 0.5192 - val_top_k_categorical_accuracy: 0.8056
Epoch 172/200
 - 27s - loss: 1.3088 - acc: 0.6469 - top_k_categorical_accuracy: 0.9103 - val_loss: 2.0915 - val_acc: 0.5226 - val_top_k_categorical_accuracy: 0.8040
Epoch 173/200
 - 27s - loss: 1.3015 - acc: 0.6510 - top_k_categorical_accuracy: 0.9111 - val_loss: 2.0882 - val_acc: 0.5272 - val_top_k_categorical_accuracy: 0.8110
Epoch 174/200
 - 27s - loss: 1.2941 - acc: 0.6526 - top_k_categorical_accuracy: 0.9151 - val_loss: 2.1705 - val_acc: 0.5160 - val_top_k_categorical_accuracy: 0.7986
Epoch 175/200
 - 27s - loss: 1.2849 - acc: 0.6538 - top_k_categorical_accuracy: 0.9153 - val_loss: 2.0990 - val_acc: 0.5226 - val_top_k_categorical_accuracy: 0.8062
Epoch 176/200
 - 27s - loss: 1.2928 - acc: 0.6513 - top_k_categorical_accuracy: 0.9140 - val_loss: 2.0928 - val_acc: 0.5278 - val_top_k_categorical_accuracy: 0.8100
Epoch 177/200
 - 27s - loss: 1.2816 - acc: 0.6574 - top_k_categorical_accuracy: 0.9151 - val_loss: 2.1681 - val_acc: 0.5140 - val_top_k_categorical_accuracy: 0.7988
Epoch 178/200
 - 27s - loss: 1.2905 - acc: 0.6560 - top_k_categorical_accuracy: 0.9138 - val_loss: 2.1553 - val_acc: 0.5166 - val_top_k_categorical_accuracy: 0.8030
Epoch 179/200
 - 27s - loss: 1.2819 - acc: 0.6587 - top_k_categorical_accuracy: 0.9133 - val_loss: 2.1590 - val_acc: 0.5204 - val_top_k_categorical_accuracy: 0.8014
Epoch 180/200
 - 27s - loss: 1.2789 - acc: 0.6569 - top_k_categorical_accuracy: 0.9156 - val_loss: 2.0667 - val_acc: 0.5310 - val_top_k_categorical_accuracy: 0.8112
Epoch 181/200
 - 27s - loss: 1.2777 - acc: 0.6561 - top_k_categorical_accuracy: 0.9161 - val_loss: 2.1262 - val_acc: 0.5216 - val_top_k_categorical_accuracy: 0.8070
Epoch 182/200
 - 26s - loss: 1.2649 - acc: 0.6617 - top_k_categorical_accuracy: 0.9179 - val_loss: 2.1434 - val_acc: 0.5202 - val_top_k_categorical_accuracy: 0.8030
Epoch 183/200
 - 27s - loss: 1.2715 - acc: 0.6583 - top_k_categorical_accuracy: 0.9171 - val_loss: 2.1660 - val_acc: 0.5118 - val_top_k_categorical_accuracy: 0.8006
Epoch 184/200
 - 27s - loss: 1.2690 - acc: 0.6599 - top_k_categorical_accuracy: 0.9171 - val_loss: 2.1977 - val_acc: 0.5086 - val_top_k_categorical_accuracy: 0.7942
Epoch 185/200
 - 27s - loss: 1.2659 - acc: 0.6594 - top_k_categorical_accuracy: 0.9191 - val_loss: 2.1677 - val_acc: 0.5140 - val_top_k_categorical_accuracy: 0.7984
Epoch 186/200
 - 27s - loss: 1.2591 - acc: 0.6636 - top_k_categorical_accuracy: 0.9197 - val_loss: 2.1005 - val_acc: 0.5282 - val_top_k_categorical_accuracy: 0.8088
Epoch 187/200
 - 27s - loss: 1.2495 - acc: 0.6631 - top_k_categorical_accuracy: 0.9181 - val_loss: 2.2122 - val_acc: 0.5064 - val_top_k_categorical_accuracy: 0.7958
Epoch 188/200
 - 27s - loss: 1.2531 - acc: 0.6612 - top_k_categorical_accuracy: 0.9202 - val_loss: 2.1586 - val_acc: 0.5104 - val_top_k_categorical_accuracy: 0.7994
Epoch 189/200
 - 27s - loss: 1.2533 - acc: 0.6647 - top_k_categorical_accuracy: 0.9193 - val_loss: 2.1382 - val_acc: 0.5186 - val_top_k_categorical_accuracy: 0.8050
Epoch 190/200
 - 27s - loss: 1.2400 - acc: 0.6650 - top_k_categorical_accuracy: 0.9221 - val_loss: 2.1849 - val_acc: 0.5178 - val_top_k_categorical_accuracy: 0.7972
Epoch 191/200
 - 27s - loss: 1.2472 - acc: 0.6658 - top_k_categorical_accuracy: 0.9209 - val_loss: 2.1913 - val_acc: 0.5132 - val_top_k_categorical_accuracy: 0.7962
Epoch 192/200
 - 27s - loss: 1.2364 - acc: 0.6677 - top_k_categorical_accuracy: 0.9215 - val_loss: 2.1657 - val_acc: 0.5182 - val_top_k_categorical_accuracy: 0.8016
Epoch 193/200
 - 27s - loss: 1.2409 - acc: 0.6666 - top_k_categorical_accuracy: 0.9216 - val_loss: 2.1959 - val_acc: 0.5176 - val_top_k_categorical_accuracy: 0.7960
Epoch 194/200
 - 27s - loss: 1.2267 - acc: 0.6688 - top_k_categorical_accuracy: 0.9248 - val_loss: 2.1580 - val_acc: 0.5172 - val_top_k_categorical_accuracy: 0.8048
Epoch 195/200
 - 27s - loss: 1.2234 - acc: 0.6693 - top_k_categorical_accuracy: 0.9242 - val_loss: 2.1294 - val_acc: 0.5226 - val_top_k_categorical_accuracy: 0.8074
Epoch 196/200
 - 27s - loss: 1.2332 - acc: 0.6643 - top_k_categorical_accuracy: 0.9240 - val_loss: 2.1050 - val_acc: 0.5256 - val_top_k_categorical_accuracy: 0.8060
Epoch 197/200
 - 27s - loss: 1.2300 - acc: 0.6682 - top_k_categorical_accuracy: 0.9226 - val_loss: 2.1339 - val_acc: 0.5296 - val_top_k_categorical_accuracy: 0.8058
Epoch 198/200
 - 27s - loss: 1.2210 - acc: 0.6730 - top_k_categorical_accuracy: 0.9249 - val_loss: 2.1992 - val_acc: 0.5118 - val_top_k_categorical_accuracy: 0.7984
Epoch 199/200
 - 27s - loss: 1.2147 - acc: 0.6733 - top_k_categorical_accuracy: 0.9260 - val_loss: 2.2605 - val_acc: 0.5034 - val_top_k_categorical_accuracy: 0.7928
Epoch 200/200
 - 27s - loss: 1.2228 - acc: 0.6717 - top_k_categorical_accuracy: 0.9243 - val_loss: 2.1434 - val_acc: 0.5196 - val_top_k_categorical_accuracy: 0.8080

Test Top 5 Accuracy: 0.898 Test Accuracy: 0.541 loss: 2.073